# Limpieza y Procesamiento

**Objetivo:** Limpiar el dataset extraído y preparar columnas clave para el análisis.

## Preparación de datos para el análisis: Popularidad vs. Puntuación  

### Limpieza y tratamiento del DataFrame

Para analizar la relación entre la popularidad y la puntuación de los videojuegos, partimos de un subconjunto del DataFrame original que incluye las siguientes columnas relevantes:

- `name`: nombre del videojuego  
- `rating`: puntuación media dada por los usuarios  
- `ratings_count`: número de valoraciones recibidas  
- `added`: número de veces que el juego ha sido añadido por usuarios a listas o favoritos  
- `released`: fecha de lanzamiento  
- `metacritic`: puntuación media otorgada por la crítica especializada

In [2]:
import pandas as pd

In [3]:
def filtro_popularidad_puntuacion(df):
    
    #  Seleccionamos las columnas relevantes para el análisis
    df_filtrado = df[['name', 'rating', 'ratings_count', 'added', 'released', 'metacritic']].copy()
    return df_filtrado

### Tratamiento de valores faltantes (`NaN`)

Para evitar eliminar videojuegos del análisis, hemos optado por **conservar todos los registros**. En lugar de eliminar filas, se ha aplicado la siguiente estrategia:

- **rating_clean**: si no hay puntuación de usuarios (`NaN`), se reemplaza por `-1`, indicando ausencia de datos.
- **added_clean**: si no hay dato de popularidad, se reemplaza también por `-1`.
- Se han añadido tres columnas booleanas auxiliares:
  - `has_rating`: indica si el juego tiene puntuación
  - `has_added`: indica si hay datos de popularidad
  - `has_metacritic`: indica si tiene puntuación metacritic

#### **En llamadas posteriores a la API las columnas `rating` y `added` no presentan `NaN`**
  No obstante dejamos la función por si en futuras llamada a la API si existen.

In [4]:
def nan_juegos(df):
    
    # Nuevas columnas para saber si hay datos reales
    df['has_rating'] = df['rating'].notna()
    df['has_added'] = df['added'].notna()
    df['has_metacritic'] = df['metacritic'].notna()

    # Rellenar NaN con valores de control
    df['rating_clean'] = df['rating'].fillna(-1)
    df['added_clean'] = df['added'].fillna(-1)

    return df


In [5]:
# Cargamos el DataFrame original en la variable df
df= pd.read_json("../data/raw/juegos_rawg.json")

In [6]:
df.head()

,slug,name,playtime,platforms,stores,released,tba,background_image,rating,rating_top,...,added_by_status.toplay,added_by_status.dropped,added_by_status.playing,esrb_rating.id,esrb_rating.name,esrb_rating.slug,esrb_rating.name_en,esrb_rating.name_ru,esrb_rating,community_rating
0,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,43,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,...,832.0,991.0,892.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
1,life-is-strange-episode-1-2,Life is Strange,6,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-01-29,False,https://media.rawg.io/media/games/562/56255381...,4.12,5,...,367.0,659.0,152.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
2,fallout-4,Fallout 4,38,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-11-09,False,https://media.rawg.io/media/games/d82/d82990b9...,3.81,4,...,439.0,1362.0,322.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
3,rocket-league,Rocket League,21,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-07-07,False,https://media.rawg.io/media/games/8cc/8cce7c0e...,3.93,4,...,114.0,1676.0,542.0,1.0,Everyone,everyone,Everyone,Для всех,NaN,NaN
4,rise-of-the-tomb-raider,Rise of the Tomb Raider,14,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-11-10,False,https://media.rawg.io/media/games/b45/b45575f3...,4.04,4,...,400.0,431.0,158.0,NaN,None,None,None,None,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 0 to 1999
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   slug                     2000 non-null   object 
 1   name                     2000 non-null   object 
 2   playtime                 2000 non-null   int64  
 3   platforms                2000 non-null   object 
 4   stores                   1989 non-null   object 
 5   released                 2000 non-null   object 
 6   tba                      2000 non-null   bool   
 7   background_image         1999 non-null   object 
 8   rating                   2000 non-null   float64
 9   rating_top               2000 non-null   int64  
 10  ratings                  2000 non-null   object 
 11  ratings_count            2000 non-null   int64  
 12  reviews_text_count       2000 non-null   int64  
 13  added                    2000 non-null   int64  
 14  metacritic               1056

In [8]:
df_pop_score = filtro_popularidad_puntuacion(df) # Filtramos el DataFrame con las columnas relevantes
df_pop_score = nan_juegos(df_pop_score) # Añadimos las columnas de control para NaN

In [9]:
df_pop_score.head()

,name,rating,ratings_count,added,released,metacritic,has_rating,has_added,has_metacritic,rating_clean,added_clean
0,The Witcher 3: Wild Hunt,4.65,6896,21662,2015-05-18,92.0,True,True,True,4.65,21662
1,Life is Strange,4.12,3706,15758,2015-01-29,83.0,True,True,True,4.12,15758
2,Fallout 4,3.81,3352,14082,2015-11-09,84.0,True,True,True,3.81,14082
3,Rocket League,3.93,2822,12769,2015-07-07,86.0,True,True,True,3.93,12769
4,Rise of the Tomb Raider,4.04,2741,12212,2015-11-10,86.0,True,True,True,4.04,12212


### Resumen de disponibilidad de datos

Antes de proceder con el análisis, es importante conocer cuántos juegos tienen realmente datos disponibles en las variables clave:

- **`rating`**: puntuación media de los usuarios.
- **`added`**: número de veces que un juego ha sido añadido por los usuarios.
- **`metacritic`**: puntuación media otorgada por la crítica.

El siguiente resumen muestra cuántos juegos tienen datos presentes y cuántos no, lo que permite entender el alcance real de los análisis posteriores y evitar sesgos derivados de valores faltantes.


In [10]:
# Juegos afectados por NaN

print(f"Total de juegos analizados: {len(df_pop_score)}\n")

print(f"Rating de usuarios:")
print(f"- Con datos: {df_pop_score['has_rating'].sum()}")
print(f"- Sin datos: {len(df_pop_score) - df_pop_score['has_rating'].sum()}\n")

print(f"Popularidad (added):")
print(f"- Con datos: {df_pop_score['has_added'].sum()}")
print(f"- Sin datos: {len(df_pop_score) - df_pop_score['has_added'].sum()}\n")

print(f"Metacritic:")
print(f"- Con datos: {df_pop_score['has_metacritic'].sum()}")
print(f"- Sin datos: {len(df_pop_score) - df_pop_score['has_metacritic'].sum()}")


Total de juegos analizados: 2000

Rating de usuarios:
- Con datos: 2000
- Sin datos: 0

Popularidad (added):
- Con datos: 2000
- Sin datos: 0

Metacritic:
- Con datos: 1056
- Sin datos: 944


In [ ]:
# Guardamos este DataFrame limpio en un archivo CSV para análisis posterior
df_pop_score.to_json("../data/processed/popularidad_vs_puntuacion.json", index=False)